In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde as kde
# from scipy.stats import norm, uniform, multivariate_normal as multinorm, norm
# from tqdm import tqdm
import seaborn as sns
%load_ext autoreload
%autoreload 2
%matplotlib inline
np.set_printoptions(edgeitems=10, linewidth=120, suppress=True, precision=8)

from pertussis import *
logger.setLevel(logging.INFO)

[ 0.0024  0.0023  0.0023  0.0022  0.0022  0.0022  0.002   0.002   0.002   0.0021
  0.0021  0.0021  0.002   0.0021  0.0021  0.0022  0.0023  0.0022  0.0022  0.0023
  0.0023  0.0023  0.0022  0.0021  0.002   0.002   0.002   0.002   0.002   0.002
  0.0019  0.0019  0.0019  0.0019  0.0018  0.0018  0.0017  0.0018  0.0018  0.0017
  0.0017  0.0018  0.0018  0.0018  0.0018  0.0018  0.0017  0.0018  0.0018  0.0018
  0.0017  0.0017  0.0017  0.0018  0.0018  0.0018  0.0018  0.0018  0.0018  0.0018]


# Make Simulation


## Init

In [2]:
# Load MCMC
mcmc = load_mcmc('./chains/1120-rho-70-multi-sigma-best-mcmc_v2_0.pkl')
print (mcmc['name'],': ',len(mcmc['chain']))
print (mcmc['tally'])
names = mcmc['names']

mcmc_v2_0 :  34820
10000


### Create subsets with effective sample size

In [3]:
# Take Subsets
take_subsets(mcmc)

Effective sample size: [ 2049.3271  1970.4551  2183.1012  2383.636 ]
[10000 10012 10024 10036 10048 10060 10072 10084 10096 10108 ..., 34708 34720
 34732 34744 34756 34768 34780 34792 34804 34816]
(2069,)
Subset length: 2051


### Create Policies

In [4]:
# Create Policies
default = init_policy('default')
everybody = init_policy('everybody', vax_ages=a_u)
no_one = init_policy('no_one', vax_ages=[])

# possible_ages = [5,6,7,8,9,10,11,12,13,15,18]

policies = [default, everybody, no_one]

# Remove
# only_age = [5,6,7,10,13]
only_age = [5,7,13]
for age in only_age:
    p_name = '{:02d}'.format(age)
    tmp_ages = (2 / 12, 4 / 12, 6 / 12, 1, age)
    tmp_policy = init_policy(p_name, vax_ages=tmp_ages)
    policies.append(tmp_policy)
    
# Shift
possible_ages = [3,4,5,6,7,13]
# possible_ages = [5,7,13]
for i in range(len(possible_ages)):
    for j in range(i):
        agej, agei = possible_ages[j],possible_ages[i]
        if (agej == 7) and (agei == 13): continue
        p_name = '{:02d},{:02d}'.format(agej,agei)
        tmp_ages = (2 / 12, 4 / 12, 6 / 12, 1, agej, agei)
        tmp_policy = init_policy(p_name, vax_ages=tmp_ages)
        policies.append(tmp_policy)

# # Add
additional_ages = [3,4,6,7,15]
# additional_ages = [5]
for age in additional_ages:
    p_name = '05,13,{:02d}'.format(age)
    tmp_ages = (2 / 12, 4 / 12, 6 / 12, 1, 5, 13, age)
    tmp_policy = init_policy(p_name, vax_ages=tmp_ages)
    policies.append(tmp_policy)
# Create the simulation


# # dynamic_add_ages = [5]
# # for age in dynamic_add_ages:
# #     for mod_year in range(4):
# #         p_name = 'DYNAMIC7_{:02d}_{:1d}'.format(age, mod_year)
# #         tmp_ages = (2 / 12, 4 / 12, 6 / 12, 1, 7, 13)
# #         tmp_policy = init_policy(p_name, vax_ages=tmp_ages, mod_year=mod_year, dynamic=age)
# #         policies.append(tmp_policy)
        
# Dynamic
dynamic_add_ages = [7]
for age in dynamic_add_ages:
    for mod_year in range(4):
        p_name = '05+13 + DYNAMIC {:02d},{:1d}'.format(age, mod_year)
        tmp_ages = (2 / 12, 4 / 12, 6 / 12, 1, 5, 13)
        tmp_policy = init_policy(p_name, vax_ages=tmp_ages, mod_year=mod_year, dynamic=age)
        policies.append(tmp_policy)

# Control for dynamic
dynamic_add_ages = [7]
for age in dynamic_add_ages:
    p_name = '05+13 + CONTROL {:02d}'.format(age)
    tmp_ages = (2 / 12, 4 / 12, 6 / 12, 1, 5, 13)
    tmp_policy = init_policy(p_name, vax_ages=tmp_ages, control=age)
    policies.append(tmp_policy)
        
print (len(policies))
simulation = init_simulation("1121-rho-70-with-dynamic", mcmc, policies)
simulation.keys()

30


dict_keys(['end', 'data_sick', 'pregnant_coverage', 'policies', 'p', 'subset_pick', 'data_hospital', 'name', 'start', 'n_policies', 'dist', 'mcmc'])

## Load

In [12]:
#load simulation
simulation = load_mcmc('./simulations/rho-60-36k.pkl')
mcmc = simulation['mcmc']
print(simulation['name'])
print(len(simulation['p']))
print(mcmc['name'])
print(len(mcmc['chain']))
print(mcmc['tally'])

rho-60-36k
5186
mcmc_v2_0
28880
10000


## Prepare

In [5]:
# Distributions
dists = mcmc['dists']

# Times
r_start = mcmc['start']
r_end = mcmc['end']
step = 1 / N

# Data
data_M, months = mcmc['datax'], mcmc['datay']
# data2, data2n, years = cases_yearly()
state_0 = mcmc['state_0']

# SIMULATE

In [10]:
simulate_future(simulation, 1000)

L 2051



In [10]:
# Save simulation
# save_mcmc(simulation, './simulations/')

'rho-60-36k'